<font size=50 color=darkblue>Min Cost Flow LP</font>

# Problem modelling in LP format

## Import necessary modules
- Import function `read_model` from DOcplex to read the Min-Cost-Flow LP from a temporary file
- Module `tempfile` is imported to create the temporary file
- Import function `remove` to delete the temporary file after reading it (required for python version < 3.12)

In [ ]:
from docplex.mp.model_reader import read_model
import tempfile
from os import remove

## Min-Cost-Flow LP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}c_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\backslash\left({\color{green}\mathcal{S}}\bigcup {\color{red}\mathcal{D}}\right)\\
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} \ge d_{(v)},\qquad&\forall v\in {\color{red}\mathcal{D}}\\
\sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} - \sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} \le s_{(v)},\qquad&\forall v\in {\color{green}\mathcal{S}}\\
0\le {\color{blue}x_{(u,v)}}\le \mu_{(u,v)},\qquad&\forall(u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/mcf.png' width=700/>

## A typical basic LP format consists of three sections
1. Objective expression, starting with a keyword for <em>objective sense</em>, which is either `Minimize` or `Maximize`
2. Constraints, starting with the keyword `Subject To`
3. Variable bounds, starting with the keyword `Bounds`
4. The LP format ends with the keyword `End`.

## Write the Min-Cost-Flow LP model (in LP format), which is assigned to the variable `mcf_str`

In [ ]:
mcf_str = '''
Minimize
 0.8 x(0,3) + 2.0 x(0,4) + 2.5 x(1,3) + 1.0 x(1,4) + 1.2 x(2,3) + 2.0 x(2,4) + 1.0 x(3,4) + 1.0 x(3,5) + 1.0 x(3,6) + 1.0 x(4,7) + 1.0 x(4,8)
 
Subject To
 flow_conservation(3): x(0,3) + x(1,3) + x(2,3) - x(3,4) - x(3,5) - x(3,6) = 0
 flow_conservation(4): x(0,4) + x(1,4) + x(2,4) + x(3,4) - x(4,7) - x(4,8) = 0
 demand_satisfaction(5): x(3,5) >= 150
 demand_satisfaction(6): x(3,6) >= 150
 demand_satisfaction(7): x(4,7) >= 150
 demand_satisfaction(8): x(4,8) >= 150
 capacity_supply(0): x(0,3) + x(0,4) <= 100
 capacity_supply(1): x(1,3) + x(1,4) <= 200
 capacity_supply(2): x(2,3) + x(2,4) <= 300

Bounds
 0 <= x(0,3) <= 300
 0 <= x(0,4) <= 300
 0 <= x(1,3) <= 300
 0 <= x(1,4) <= 300
 0 <= x(2,3) <= 300
 0 <= x(2,4) <= 300
 0 <= x(3,4) <= 300
 0 <= x(3,5) <= 300
 0 <= x(3,6) <= 300
 0 <= x(4,7) <= 300
 0 <= x(4,8) <= 300
End
'''

## To import the LP model to DOcplex
- Store the model to a temporary file
- Have DOcplex read the model from the temporary file

In [ ]:
# Store string to tmp file
with tempfile.TemporaryFile(delete=False) as tmp:
    tmp.write(mcf_str.encode('utf-8'))
    tmp.close()
    # Have DOcplex read the string
    mcf_LP = read_model(filename=tmp.name, model_name='Min Cost Flow')
    # Delete the temporary file
    remove(tmp.name)

## Summarize the model

In [ ]:
mcf_LP.print_information()

## Solve the LP and display the result

In [ ]:
mcf_sol = mcf_LP.solve()
if mcf_sol:
    mcf_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`, `re`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import re

## Extract the link list `E`, link capacities $\mu$, supply node list <code><font color='green'>S</font></code>, and demand node list <code><font color='red'>D</font></code> from the LP model

In [ ]:
E, mu, S, D = [], {}, [], []

for var in mcf_LP.iter_variables():
    u, v = re.findall('x\((.+),(.+)\)', var.name)[0]
    E.append(e:=(int(u), int(v)))
    mu[e] = int(var.ub)

for c in mcf_LP.iter_constraints():
    if c.name.startswith('capacity'):
        v = re.findall('capacity_supply\((.+)\)', c.name)[0]
        S.append(int(v))
    elif c.name.startswith('demand'):
        v = re.findall('demand_satisfaction\((.+)\)', c.name)[0]
        D.append(int(v))

## For convenience, extract the solution to a dictionary named `sol_x`

In [ ]:
sol_x = {(u,v): mcf_sol.get_value(f'x({u},{v})') for u,v in E}
sol_x

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[S]['color'] = 'green'
g.vs[D]['color'] = 'red'

g.es['flow'] = [sol_x[e] for e in E]
g.es['cap'] = [mu[e] for e in E]
g.es['label'] = ' '
g.es['arrow_size'] = [6 + 4*(sol_x[e] > 0) for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [.5 + 2.5*(sol_x[e] > 0) for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['flow_label_color'] = ['blue' if sol_x[e] > 0 else 'black' for e in E]
g.es['cap_label_color']  = ['red' if sol_x[e] > 0 else 'black' for e in E]
g.es['label_size'] = [7 + 3*(sol_x[e] > 0) for e in E]

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('tree'), edge_label=g.es['label'], target=ax)

for e, edge_label in zip(g.es, p.get_edge_labels()):
    txt = ax.annotate(f'{e["flow"]:g}', xycoords=edge_label, xy=(-5,0), color=e['flow_label_color'], backgroundcolor='white', size=e['label_size'])
    txt =  ax.annotate(f' | ', xycoords=txt, xy=(1,0), color='k', size=e['label_size'], verticalalignment='bottom')
    txt =  ax.annotate(f'{e["cap"]}', xycoords=txt, xy=(1,0), color=e['cap_label_color'], backgroundcolor='white', size=e['label_size'], verticalalignment='bottom')
    
plt.show()